<a href="https://colab.research.google.com/github/cypox/rl-trading-bot/blob/main/rl_trading_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# USE A CLONED REPO!
#!git clone https://github.com/pskrunner14/trading-bot.git bot
#%cd bot/
#!pip3 install -r requirements.txt
#!python3 train.py data/GOOG.csv data/GOOG_2018.csv --strategy t-dqn
#!python3 eval.py data/GOOG_2019.csv --model-name model_double-dqn_GOOG_50 --debug

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

Helper functions

In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def stock_price_format(n):
  if n < 0:
    return "- # {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

def stocks_price_format(n):
  if n < 0:
    return "- # {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

def dataset_loader(stock_name, source="yahoo"):
  dataset = data_reader.DataReader(stock_name, data_source=source)
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[1]).split()[0]
  
  close = dataset['Close']
  
  return close

def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = -starting_id * [data[0]] + list(data[0:timestep+1])

  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

Loading Apple stocks data

In [ ]:
stock_name = "AAPL"
source = "yahoo"
data = dataset_loader(stock_name, source)

Setting model configuration

In [ ]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

Defining our Deep Q-Learning Trader

In [ ]:
class AI_Trader():

  def __init__(self, state_size, action_space=3, model_name="AITrader"):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    self.model = self.model_builder()
  
  def model_builder(self):
        model = tf.keras.models.Sequential()
        #model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(tf.keras.layers.LSTM(units=16, activation='relu', recurrent_activation='relu', input_shape=(self.state_size, 1)))
        model.add(tf.keras.layers.Dense(units=16, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
        return model

  def trade(self, state):
      if random.random() <= self.epsilon:
          return random.randrange(self.action_space)
      actions = self.model.predict(state)

  def batch_train(self, batch_size):
      batch = []
      for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
        batch.append(self.memory[i])
      for state, action, reward, next_state, done in batch:
        reward = reward
        if not done:
          reward = reward + self.gamma * np.amax(self.model.predict(np.expand_dims(next_state, 2))[0])
        target = self.model.predict(np.expand_dims(state, 2))
        target[0][action] = reward
        self.model.fit(np.expand_dims(state, 2), target, epochs=1, verbose=0)
      if self.epsilon > self.epsilon_final:
        self.epsilon *= self.epsilon_decay

In [ ]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 16)                1152      
_________________________________________________________________
dense_22 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_23 (Dense)             (None, 3)                 51        
Total params: 1,475
Trainable params: 1,475
Non-trainable params: 0
_________________________________________________________________


Training

In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(np.expand_dims(state, 2))
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
    
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))


  0%|          | 0/1257 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 30.142500
AI Trader bought:  $ 29.192499
AI Trader sold:  $ 29.027500  Profit: - # 1.115000
AI Trader sold:  $ 28.930000  Profit: - # 0.262499
AI Trader bought:  $ 29.322500
AI Trader sold:  $ 29.695000  Profit: $ 0.372499
AI Trader bought:  $ 29.825001
AI Trader sold:  $ 29.437500  Profit: - # 0.387501
AI Trader bought:  $ 29.719999
AI Trader sold:  $ 29.575001  Profit: - # 0.144999
AI Trader bought:  $ 28.799999
AI Trader bought:  $ 29.757500
AI Trader sold:  $ 29.570000  Profit: $ 0.770000
AI Trader bought:  $ 29.557501
AI Trader sold:  $ 28.295000  Profit: - # 1.462500
AI Trader bought:  $ 28.120001
AI Trader bought:  $ 27.834999
AI Trader sold:  $ 27.245001  Profit: - # 2.312500
AI Trader bought:  $ 26.507500
AI Trader sold:  $ 26.832500  Profit: - # 1.287500
AI Trader bought:  $ 27.152500
AI Trader bought:  $ 27.007500



  3%|▎         | 35/1257 [00:10<35:45,  1.76s/it]

AI Trader sold:  $ 26.830000  Profit: - # 1.004999



  3%|▎         | 36/1257 [00:14<46:31,  2.29s/it]

...

AI Trader bought:  $ 29.237499



 22%|██▏       | 282/1257 [14:00<54:02,  3.33s/it]

AI Trader sold:  $ 29.264999  Profit: $ 2.904999



 23%|██▎       | 285/1257 [14:10<53:13,  3.29s/it]

KeyboardInterrupt: ignored